In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName('flights').getOrCreate()

23/01/08 11:06:40 WARN Utils: Your hostname, MacBook-Air-di-Teodoro.local resolves to a loopback address: 127.0.0.1; using 192.168.240.184 instead (on interface en0)
23/01/08 11:06:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/08 11:06:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# count the missing values in each column
def count_null(df):
    cols_check = df.columns
    df.select(*[
    (
        F.count(F.when((F.isnan(c) | F.col(c).isNull()), c)) if t not in ("timestamp", "date")
        else F.count(F.when(F.col(c).isNull(), c))
    ).alias(c)
    for c, t in df.dtypes if c in cols_check
]).show()

In [6]:
# extract the selected features from file
def select_features():
    features = []
    with open("../../util/features_analysis.txt","r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            features.append(line)
    return features


In [7]:
select_features()

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'Origin',
 'Dest',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Distance',
 'DistanceGroup',
 'DivAirportLandings',
 'DestCityName',
 'OriginCityName']

In [8]:
# select the features from the dataset and drop the missing values
def clean_data(df,features):
    df = df.select(features)
    df = df.na.drop()
    return df

In [9]:
# complete the geographical information with latitude and longitude
def join_airports(df,airports):
    df = df.join(airports,df.Origin==airports.IATA)
    df = df.withColumnRenamed("LATITUDE","ORIGIN_LATITUDE")
    df = df.withColumnRenamed("LONGITUDE","ORIGIN_LONGITUDE")
    df = df.withColumnRenamed("STATE","ORIGIN_STATE")
    df = df.withColumnRenamed("AIRPORT", "ORIGIN_AIRPORT_FULL_NAME")
    df = df.withColumnRenamed("STATE_FULL_NAME", "ORIGIN_STATE_FULL_NAME")
    df = df.drop("IATA","CITY","COUNTRY")
    df = df.join(airports,df.Dest==airports.IATA)
    df = df.withColumnRenamed("LATITUDE","DEST_LATITUDE")
    df = df.withColumnRenamed("LONGITUDE","DEST_LONGITUDE")
    df = df.withColumnRenamed("STATE","DEST_STATE")
    df = df.withColumnRenamed("AIRPORT", "DEST_AIRPORT_FULL_NAME")
    df = df.withColumnRenamed("STATE_FULL_NAME", "DEST_STATE_FULL_NAME")
    df = df.drop("IATA","CITY","COUNTRY")    
    return df

In [10]:
# get all files with .csv extension
import glob
files = glob.glob("../../data.nosync/*.csv")

In [11]:
# get the first file and use it to infer the schema
df_tmp = spark.read.csv(files[0],header=True,inferSchema=True)

In [12]:
schema = df_tmp.schema

In [13]:
# save schema as json
with open("schema.json","w") as f:
    f.write(schema.json())


In [14]:
# load schema from json
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
with open("schema.json","r") as f:
    schema_loaded = StructType.fromJson(json.load(f))


In [16]:
from pyspark.sql.functions import date_format, col, concat_ws, to_date, unix_timestamp

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

def preprocess_normal_data(path):
    df = spark.read.csv(path, schema=schema, header=True)
    columns = df.columns
    df_normal = df.filter(df['Diverted'] == 0)
    features = select_features()
    df_normal = clean_data(df_normal,features)
    airports = spark.read.csv("../../util/airports_fil.csv", inferSchema=True, header=True)
    df_normal = join_airports(df_normal,airports)
    df_normal =  df_normal.withColumn("WeekofMonth", concat_ws("-", df_normal.Year, df_normal.Month, date_format(col("FlightDate"), "W").cast("string")))
    df_normal = df_normal.withColumn("WeekofMonth", to_date(unix_timestamp(col('WeekofMonth'), 'yyyy-MM-dd').cast("timestamp")))
    return df_normal

In [17]:
# obtain the preprocessed data
df_cleaned = preprocess_normal_data("../../data.nosync")
df_cleaned = df_cleaned.drop("_c0")

In [18]:
df_cleaned.columns

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'Origin',
 'Dest',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Distance',
 'DistanceGroup',
 'DivAirportLandings',
 'DestCityName',
 'OriginCityName',
 'ORIGIN_AIRPORT_FULL_NAME',
 'ORIGIN_STATE',
 'ORIGIN_LATITUDE',
 'ORIGIN_LONGITUDE',
 'ORIGIN_STATE_FULL_NAME',
 'DEST_AIRPORT_FULL_NAME',
 'DEST_STATE',
 'DEST_LATITUDE',
 'DEST_LONGITUDE',
 'DEST_STATE_FULL_NAME',
 'WeekofMonth']

In [19]:
# save the cleaned data
df_cleaned.write.csv("../../data.nosync/cleaned/cleaned_flights.csv",header=True)

23/01/08 11:09:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
df_cleaned.count()